In [1]:
import jovian
import requests
from bs4 import BeautifulSoup
import pandas as pd
from PIL import Image
from pytesseract import pytesseract
from collections import defaultdict

<IPython.core.display.Javascript object>

In [2]:
# download the web page using requests
book_info = 'https://www.goodreads.com/book/show/2767052-the-hunger-games'
header = {'User_Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
response = requests.get(book_info,headers=header)

# capture the text content of the page from response
page_contents = response.text

In [3]:
# Save the page as html to avoid scrapping multiple times while experimenting
with open('books.html', 'w', encoding="utf-8") as f:
    f.write(page_contents)

# read it html page to scrape information
with open('books.html', 'r', encoding="utf-8") as f:
    html_source = f.read()

# The BeautifulSoup returns a document which we shall use to extract information
doc_book = BeautifulSoup(html_source, 'html.parser')

In [4]:
keyList = ["Title",
            "Author",
            "About_author",
            "Foreword",
            "Description",
            "Image",
            "Genres", 
            "Series",
            "Order",
            "Pages",
            "Format",
            "First_published",
            "Rating",
            "Review_count"]

In [5]:
books = []
book = {key: None for key in keyList}

In [6]:
# get all the company name in the page
book_name = doc_book.find('h1', attrs={'data-testid': 'bookTitle'})
#print(book_name)
book['Title'] = book_name.text.strip()
# for the first company name, get the exact name
print(book['Title'])

The Hunger Games


In [7]:
book_author = doc_book.find('span', attrs={'data-testid': 'name'})
#print(book_author)
book['Author'] = book_author.text.strip()
# for the first company name, get the exact name
print(book['Author'])

Suzanne Collins


In [8]:
book_rating = doc_book.find('div', class_= 'RatingStatistics__rating')
#print(book_rating)
book['Rating'] = book_rating.text.strip()
print(book['Rating'])

4.33


In [9]:
review_count = doc_book.find('span', attrs={'data-testid': 'reviewsCount'})
#print(review_count)
review_count = review_count.text.strip()
review_count = ''.join(x for x in review_count if x.isdigit())

In [10]:
book['Review_count'] = review_count
# for the first company name, get the exact name
print(book['Review_count'])

205790


In [11]:
book_description = doc_book.find('div', attrs={'data-testid': 'description'})
#print(book_description)
br = book_description.find('b')
if br != None:
    book['Foreword'] = br.text.strip()
    print(book['Foreword'])
    book_description.find('b').decompose()

Could you survive on your own in the wild, with every one out to make sure you don't live to see the morning?


In [12]:
book_description = doc_book.find('div', attrs={'data-testid': 'description'})
book_description

<div class="BookPageMetadataSection__description" data-testid="description"><div class="TruncatedContent" tabindex="-1"><div class="TruncatedContent__text TruncatedContent__text--large" data-testid="contentContainer" tabindex="-1"><div class="DetailsLayoutRightParagraph"><div class="DetailsLayoutRightParagraph__widthConstrained"><span class="Formatted"><br/><br/>In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and one girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.<br/><br/>Sixteen-year-old Katniss Everdeen, who lives alone with her mother and younger sister, regards it as a death sentence when she steps forward to take her sister's place in the Games. But Katniss has been close to dead before—and survival, for her, is second natu

In [13]:
des = book_description.get_text(separator = '\n', strip = True)
book['Description'] = des
book['Description']

"In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and one girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.\nSixteen-year-old Katniss Everdeen, who lives alone with her mother and younger sister, regards it as a death sentence when she steps forward to take her sister's place in the Games. But Katniss has been close to dead before—and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against humanity and life against love."

In [14]:
book_image = doc_book.find('img', class_='ResponsiveImage')
book['Image'] = book_image['src']
book['Image']

'https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1586722975i/2767052.jpg'

In [25]:
genreFile = open('genres.txt', "w")
book['Genres'] = []
genres = []
for type in doc_book.find_all('span', class_='BookPageMetadataSection__genreButton'):
    genre = type.text.strip()
    genres = ",".join(genre)
    a = type.find('a')
    a = str(a.get('href'))
    genreFile.write(a+"\n")
    genres.append(genre)
    book['Genres'].extend([genre])
print(book['Genres'])
genreFile.close()

'ARSDFFY,o,u,n,g, ,A,d,u,l,tiY,o,u,n,g, ,A,d,u,l,tcY,o,u,n,g, ,A,d,u,l,ttY,o,u,n,g, ,A,d,u,l,tiY,o,u,n,g, ,A,d,u,l,toY,o,u,n,g, ,A,d,u,l,tnaFY,o,u,n,g, ,A,d,u,l,tiY,o,u,n,g, ,A,d,u,l,tcY,o,u,n,g, ,A,d,u,l,ttY,o,u,n,g, ,A,d,u,l,tiY,o,u,n,g, ,A,d,u,l,toY,o,u,n,g, ,A,d,u,l,tnnFY,o,u,n,g, ,A,d,u,l,tiY,o,u,n,g, ,A,d,u,l,tcY,o,u,n,g, ,A,d,u,l,ttY,o,u,n,g, ,A,d,u,l,tiY,o,u,n,g, ,A,d,u,l,toY,o,u,n,g, ,A,d,u,l,tntFY,o,u,n,g, ,A,d,u,l,tiY,o,u,n,g, ,A,d,u,l,tcY,o,u,n,g, ,A,d,u,l,ttY,o,u,n,g, ,A,d,u,l,tiY,o,u,n,g, ,A,d,u,l,toY,o,u,n,g, ,A,d,u,l,tnaFY,o,u,n,g, ,A,d,u,l,tiY,o,u,n,g, ,A,d,u,l,tcY,o,u,n,g, ,A,d,u,l,ttY,o,u,n,g, ,A,d,u,l,tiY,o,u,n,g, ,A,d,u,l,toY,o,u,n,g, ,A,d,u,l,tnsFY,o,u,n,g, ,A,d,u,l,tiY,o,u,n,g, ,A,d,u,l,tcY,o,u,n,g, ,A,d,u,l,ttY,o,u,n,g, ,A,d,u,l,tiY,o,u,n,g, ,A,d,u,l,toY,o,u,n,g, ,A,d,u,l,tnyyFFY,o,u,n,g, ,A,d,u,l,tiY,o,u,n,g, ,A,d,u,l,tcY,o,u,n,g, ,A,d,u,l,ttY,o,u,n,g, ,A,d,u,l,tiY,o,u,n,g, ,A,d,u,l,toY,o,u,n,g, ,A,d,u,l,tnaFY,o,u,n,g, ,A,d,u,l,tiY,o,u,n,g, ,A,d,u,l,tcY,o,u,n,g

In [29]:
book['Genres'] = []
genres = []
for type in doc_book.find_all('span', class_='BookPageMetadataSection__genreButton'):
    genre = type.text.strip()
    genres.append(genre)

genre = ", ".join(genres)
print(genre)

Young Adult, Fiction, Fantasy, Dystopia, Science Fiction, Romance, Adventure


In [16]:
seriesFile = open('series.txt', "w")
book_series = doc_book.find('h3', class_='Text Text__title3 Text__italic Text__regular Text__subdued')
if book_series != None:
    a = book_series.find('a')
    series_link = a.get('href')
    seriesFile.write(series_link)
    series = book_series.text.strip().split('#')
    book['Series'] = series[0]
    book['Order'] = series[1]
seriesFile.close()

In [17]:
book_form = doc_book.find('p', attrs={'data-testid': 'pagesFormat'})
#print(book_form)
book_form = book_form.text.strip()
book_form = book_form.split(' ')
book['Pages'] = book_form[0]
book['Format'] = book_form[2]
# for the first company name, get the exact name
print(book['Pages'])
print(book['Format'])

374
Hardcover


In [18]:
book_date = doc_book.find('p', attrs={'data-testid': 'publicationInfo'})
book_date = book_date.text.strip()
book_date = book_date.split(' ')
book_date.remove('First')
book_date.remove('published')
book['First_published'] = ' '.join(book_date)
book['First_published']

'September 14, 2008'

In [19]:
authorFile = open("authors.txt", "w") 
author_info = doc_book.find('div', class_='FeaturedPerson__infoPrimary')
author_info = author_info.find('a')
author_info = author_info.get('href')
authorFile.write(author_info)
authorFile.close()


In [20]:
reviewFile = open("reviews.txt", "w") 
reviews = []
reviews = doc_book.find_all('span', class_='Text Text__body3')
for i in reviews:
    review = i.find('a')
    if review is None:
        continue
    else:
        review = str(review.get('href'))
        reviewFile.write(review+"\n")
reviewFile.close()

In [21]:
author_des = doc_book.find_all('div', class_='DetailsLayoutRightParagraph__widthConstrained')
author_des = author_des[1]
i = author_des.find('i')
if i != None:
    author_des.find('i').decompose()
author_des = author_des.get_text(separator='\n', strip=True)
book['About_author'] = author_des
book['About_author']

'Since 1991, Suzanne Collins has been busy writing for children’s television. She has worked on the staffs of several Nickelodeon shows, including the Emmy-nominated hit Clarissa Explains it All and The Mystery Files of Shelby Woo. For preschool viewers, she penned multiple stories for the Emmy-nominated Little Bear and Oswald. She also co-wrote the critically acclaimed Rankin/Bass Christmas special, Santa, Baby! Most recently she was the Head Writer for Scholastic Entertainment’s Clifford’s Puppy Days.\nWhile working on a Kids WB show called Generation O! she met children’s author James Proimos, who talked her into giving children’s books a try.\nThinking one day about Alice in Wonderland, she was struck by how pastoral the setting must seem to kids who, like her own, lived in urban surroundings. In New York City, you’re much more likely to fall down a manhole than a rabbit hole and, if you do, you’re not going to find a tea party. What you might find...? Well, that’s the story of Gre

In [22]:
book

{'Title': 'The Hunger Games',
 'Author': 'Suzanne Collins',
 'About_author': 'Since 1991, Suzanne Collins has been busy writing for children’s television. She has worked on the staffs of several Nickelodeon shows, including the Emmy-nominated hit Clarissa Explains it All and The Mystery Files of Shelby Woo. For preschool viewers, she penned multiple stories for the Emmy-nominated Little Bear and Oswald. She also co-wrote the critically acclaimed Rankin/Bass Christmas special, Santa, Baby! Most recently she was the Head Writer for Scholastic Entertainment’s Clifford’s Puppy Days.\nWhile working on a Kids WB show called Generation O! she met children’s author James Proimos, who talked her into giving children’s books a try.\nThinking one day about Alice in Wonderland, she was struck by how pastoral the setting must seem to kids who, like her own, lived in urban surroundings. In New York City, you’re much more likely to fall down a manhole than a rabbit hole and, if you do, you’re not goi